<a href="https://colab.research.google.com/github/jkim2260/CoworkPractice/blob/master/FL_LSTM_Finish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Library 불러오기
import torch
import torch.nn as nn
import torch.functional as F
import torch.autograd as Variable   
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torch.utils.data as data_utils

# Python Library 불러오기
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as mlt
import seaborn as sns

#sklearn 라이브러리 불러오기
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler  #정규화 과정
from sklearn.model_selection import train_test_split #데이터 분리

#이미지 시각화
import matplotlib.pyplot as plt


#진행사항 파악
import time
from tqdm.auto import tqdm

#경고 끄기
import warnings
warnings.filterwarnings('ignore')


#Check GPU and fix the random seed
#쿠다 연동가능한지 중간중간 코드에 넣는 방법 파악.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#재현성을 결정하는 단계]\
#넘파이, 토치, cuda 랜덤시드 고정(똑같은 값을 비교를 위해 가중치 값이 랜덤으로 변화하는 것을 방지하는 목적) 
#향후 모든텐서를 gpu에 넣어줌, 모델을 gpu에 넣어줌
np.random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#전처리된 데이터 읽기
df= pd.read_csv('C:/Users/gk571/OneDrive/바탕 화면/FL/ISE_537_4Buildings_Final.csv', encoding='utf-8')
#df.set_index('DateTime', inplace=True) #데이터 프레임의 인덱스를 datetime으로 설정
print(df)

#get the numpy data
train_data = df.iloc[:,:38].values

#Feature engineering
def sliding_windows(data, seq_length):
  X = [ ]
  y = [ ]

  for i in range(len(data)-seq_length-1):
      _X = data[i:(i+seq_length)]
      _y = data[i + seq_length]
      X.append(_X)
      y.append(_y)
  return np.array(X), np.array(y)

# Feature engineering
sc = MinMaxScaler()
train_data = sc.fit_transform(train_data)
seq_length = 24

x, y = sliding_windows(train_data, seq_length)
train_len = int(len(y) * 0.7)
valid_len = len(y) - train_len
x_tensor = torch.tensor(x, dtype=torch.float) #.cuda()
y_tensor = torch.tensor(y, dtype=torch.float) #.cuda()
train_x = x_tensor[:train_len] #.cuda()
train_y = y_tensor[:train_len] #.cuda()
valid_x = x_tensor[train_len:] #.cuda()
valid_y = y_tensor[train_len:] #.cuda()

#Training data batch learning(배치넣을려고 들고오는 것)
train_data = data_utils.TensorDataset(train_x, train_y)
dataloader = data_utils.DataLoader(train_data, batch_size = 10, shuffle = False) #시계열 데이터는 shuffle=False #drop_last= True???

class LSTM(nn.Module):
    def __init__(self, num_features, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_features = num_features
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
        num_layers=num_layers, batch_first=True) #.cuda()
        self.fc = nn.Linear(hidden_size, num_features) #.cuda()
    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #.cuda()
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #.cuda()
        # Propagate input through LSTM
        _, (h_out, _) = self.lstm(x, (h_0, c_0)) #.cuda()
        h_out = h_out.view(-1, self.hidden_size) #.cuda() # (batch_size * num_layers, hidden_size)
        out = self.fc(h_out) #.cuda()
        return out
    
num_epochs = 2000
learning_rate = 0.01
input_size = 38
hidden_size = 10
num_layers = 1
num_features = 1

lstm = LSTM(num_features, input_size, hidden_size, num_layers)
#lstm.cuda()
#lstm = lstm.to(device)
#lstm = lstm.to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

def CVRMSE(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


RMSE = []
loss = []
for epoch in range(num_epochs):
    for idx, (x_batch, y_batch) in enumerate(dataloader):
        outputs = lstm(train_x)
        # obtain the loss function
        loss = criterion(outputs, train_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        RMSE = CVRMSE(train_y.detach().numpy(), outputs.detach().numpy())
        if epoch % 100 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.detach().item()))
            #print("Epoch: %d, (RMSE))

RMSE.append(RMSE.item())
loss.append(loss.item())
        

#Visualize the actual data and predictions
lstm.eval()
with torch.no_grad():
    train_predict = lstm(valid_x)

pred_y = train_predict.detach().numpy()
act_y = y_tensor.detach().numpy()
pred_y = sc.inverse_transform(pred_y)
act_y = sc.inverse_transform(act_y)

plt.axvline(x=train_len, c='r', linestyle='--')
plt.plot(act_y, label='actual')
plt.plot(pred_y, label='predicted')
plt.legend()
plt.suptitle('Time-Series Prediction')
plt.show()

plt.figure(figsize=(16,9))
plt.title("loss value of LSTM")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(loss, label='Centralized Learning')
plt.legend()
plt.show()

plt.figure(figsize=(16,9))
plt.title("CVRMESE trend of LSTM")
plt.xlabel("epoch")
plt.ylabel("CVRMSE")
plt.plot(RMSE, label='Centralized Learning')
plt.legend()
plt.show()